In [25]:
import pandas as pd
from datetime import datetime

intraday = pd.read_csv("../00_data/[backup] intraday.csv", sep=";", index_col=["date", "hours"], decimal=",")
day_ahead = pd.read_csv("../00_data/[backup] day_ahead_auction_longformat.csv", sep=";", decimal=",")
forecasts = pd.read_csv("../00_data/[backup] wind_and_solar--actual_and_forecast--capacity_and_profile.csv", sep=";", decimal=".")

#print(intraday.head(5))
#print(day_ahead.head(5))
#print(forecasts.head(2))

forecasts["time"] = forecasts["cet_cest_timestamp"].str[:-5] #cut +... part
forecasts["timestamp"] = forecasts["time"].map(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S'))
forecasts["solar_forecast_error"] = forecasts["solar_forecast_error"]*(-1)
forecasts["wind_forecast_error"] = forecasts["wind_forecast_error"]*(-1)

intraday = intraday.iloc[:, 3:4] # use only weight. avg column for intraday
merged_data = intraday.merge(day_ahead, left_on=["date", "hours"], right_on=["date", "hour"])
merged_data = merged_data[["date", "hour", "dayAheadPrice", "Weight. Avg.(€/MWh)"]]
merged_data.columns = ["date", "hour", "day_ahead_price", "intraday_price"]
merged_data["price_diff"] = merged_data["day_ahead_price"]-merged_data["intraday_price"]
merged_data["hour"] = merged_data["hour"].str[1:3]
merged_data["time"] = merged_data[['date', 'hour']].apply(lambda x: ' '.join(x), axis=1)
merged_data["datetime"] = merged_data["time"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H'))

merged_data = merged_data.merge(forecasts, left_on=["datetime"], right_on=["timestamp"])
merged_data = merged_data[['date','hour', 'datetime', 'day_ahead_price','intraday_price','price_diff','solar_capacity','solar_generation_actual','wind_capacity','wind_generation_actual','wind_generation_forecast', 'solar_generation_forecast']]
# zeitumstellung => leere Zeilen löschen
merged_data = merged_data[(merged_data.datetime != "2016-03-27 02:00:00") & (merged_data.datetime != "2017-03-26 02:00:00")]

#print(merged_data)
merged_data.to_csv("../00_data/merged_all_by_algo.csv", sep=";", decimal=",", index=False)
print("done")

done
